In [ ]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 62.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=ffd047bd3fa9784f674a3bf822d49f02c811e10942484e09077dd6df292dc498
  Stored in directory:

In [ ]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2lxpqNk7u6Ap5Z9GaqQEj4psoNL_2HNfWPdbK1A6Bg1aP4og9

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
pinecone = Pinecone(api_key='d130d359-d4c6-4ce9-b5c8-4e25368899fe')
INDEX_NAME = 'shoe'
index = pinecone.Index(INDEX_NAME)

def pick_shoe_items(shoe):
  result = {}
  result['id']    = shoe.get('id')
  result['name']  = shoe.get('metadata').get('name')
  result['image'] = shoe.get('metadata').get('image')
  result['price'] = shoe.get('metadata').get('price') # shoe['price'] is same as shoe.get('price')
  # Now try to also set a nice looking message for the shoe from AI
  # chat completion

  return result

def run_query(query):
  results = []
  embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
  embedding = embedding_model.encode(query).tolist()
  query_response = index.query(top_k=5, vector=embedding, include_metadata=True, include_values=True)
  results = map(pick_shoe_items,query_response['matches'])
  # result_names = [result['metadata']['name'] for result in results['matches']]
  # result_urls = [result['metadata'].get('imageURLs', 'URL not available') for result in results['matches']]
  # result= {'names': result_names, 'imageURLs': result_urls}
  return list(results)
  # return query_response

In [ ]:
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/api/shoes')
def home():
    query = request.args.get('q')
    r= run_query(query)
    return jsonify(r)

# Open an ngrok tunnel to the port where your Flask app is running
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
# Run Flask app (this will block execution, so ensure ngrok is set up first)
app.run()

Public URL: NgrokTunnel: "https://b74a-34-31-235-122.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Sep/2024 10:14:50] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Sep/2024 10:14:51] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /api/shoes [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])